# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [16]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option")
for item in movie_item:
    if item.text != "請選擇片名":
        print(item.text+' : '+item['value'])
#for p in movie_item:
    #print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

導演先生的完美假期 : 10545
大地蜜語 : 10513
狂飆一夢 : 10506
親愛的莎瑪 : 10491
大畫特務 : 10478
戰慄糖果屋 : 10472
失戀殺人：三角禁戀 : 10465
明明會說話 : 10440
艾瑪 : 10396
罪樂園 : 10349
寄生上流：黑白版 : 10527
千日千夜 : 10490
咒怨2 電影版 : 10464
鳴鳥不飛：烏雲密布 : 10458
哥布林殺手 劇場版 : 10438
女鬼橋 : 10437
怨咒 : 10421
娘娘腔日記 : 10401
汪星人的奇幻漂流 : 10388
極地守護犬 : 10383
叔．叔 : 10371
隱形人 : 10359
失路人 : 10498
電影音效傳奇：好萊塢之聲 : 10484
悲慘世界 : 10479
吹哨奇案 : 10475
閃閃小超人電影版 : 10474
少年阿罕默德 : 10468
咒怨 電影版 : 10463
陰櫥 : 10457
絕命大平台 : 10447
李察朱威爾事件 : 10368
音速小子 : 9246
你好．愛．再見 : 10501
逃出夢幻島 : 10485
雨天．紐約 : 10476
超＂人＂氣動物園 : 10445
就愛斷捨離 : 10436
愛情人形 : 10394
蜜蜂與遠雷 : 10419
電影版 輝夜姬想讓人告白～天才們的戀愛頭腦戰～ : 10405
古曼童 : 10399
猛禽小隊：小丑女大解放 : 10268
紅鞋公主與七矮人 : 10417
1917 : 10189
你的情歌 : 10407
杜立德 : 10316
她們 : 10202
變身特務 : 9008
巧虎電影：英雄遊樂園 : 10429
青春催落去 : 10404
絕地戰警FOR LIFE : 10242
星星知我心 : 10409
逃出天堂島 : 10384
魯邦三世 The First : 10355
別告訴她 : 10382
真愛裁會贏 : 10362
潛艦追緝 : 10333
霹靂嬌娃 : 10124
CATS貓 : 10166
STAR WARS : 天行者的崛起 : 10225
復仇母親 : 10392
8級警戒 : 10323
黑色聖誕節 : 10291
去年聖誕節 : 10256
野蠻遊戲：全面晉級 : 10134
寂寞診療室 : 1028

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [17]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10169

In [21]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [11]:
# 指定放映地區
area_id = 28

In [12]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

八月 21
八月 22
八月 23
八月 24
八月 25


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [13]:
# 選定要觀看的日期
date = "2019-08-21"

In [14]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [16]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''


----------------------------------------------------------------------
電影院:  台北美麗華大直影城
放映類型:  數位
2019-08-21 09:00:00
2019-08-21 11:10:00
2019-08-21 13:15:00
2019-08-21 15:20:00
2019-08-21 19:30:00
2019-08-21 21:40:00
2019-08-21 22:30:00
----------------------------------------------------------------------
電影院:  台北新光影城
放映類型:  數位
2019-08-21 10:00:00
2019-08-21 14:50:00
2019-08-21 19:30:00
----------------------------------------------------------------------
電影院:  台北in89豪華數位影城
放映類型:  數位
2019-08-21 09:30:00
2019-08-21 11:20:00
2019-08-21 13:15:00
2019-08-21 15:10:00
2019-08-21 16:10:00
2019-08-21 17:10:00
2019-08-21 18:05:00
2019-08-21 19:10:00
2019-08-21 21:10:00
2019-08-21 23:10:00
2019-08-22 01:10:00
----------------------------------------------------------------------
電影院:  台北日新威秀影城
放映類型:  數位
2019-08-21 09:00:00
2019-08-21 10:55:00
2019-08-21 12:50:00
2019-08-21 14:45:00
2019-08-21 16:40:00
2019-08-21 18:35:00
2019-08-21 20:35:00
-----------------------------------------------------